##Classificação de dados utilizando redes neurais

O objetivo do problema consiste em aplicar uma rede neural para classificar dados de uma base de dados de vinhos.


####Módulos

In [3]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [64]:
def get_data() -> pd.DataFrame:
    """
    Realiza o download dos arquivos .csv, 
    concatena-os e renomeia os atributos 
    de inglês para português

    Retorna:
    - df: DataFrame da base de dados
    """
    
    path_1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/" \
                "wine-quality/winequality-red.csv"

    path_2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/" \
                "wine-quality/winequality-white.csv"

    df1 = pd.read_csv(path_1, delimiter=";")
    df1["Tipo"] = "Tinto"

    df2 = pd.read_csv(path_2, delimiter=";")
    df2["Tipo"] = "Branco"

    df = pd.merge(df1,df2, how="outer")

    dict = {'fixed acidity':'Acidez fixa',
            'volatile acidity':'Acidez volátil',
            'citric acid':'Ácido cítrico',
            'residual sugar':'Açúcar residual',
            'chlorides':'Cloretos',
            'free sulfur dioxide':'Dióxido de enxofre livre',
            'total sulfur dioxide':'Dióxido de enxofre total',
            'density':'Densidade',
            'sulphates':'Sulfatos',
            'alcohol':'Álcool',
            'quality':'Qualidade'}

    df.rename(columns=dict, inplace=True)
    
    return df

###Divisão da base e treinamento de modelos

In [ ]:
def apply_stratified_holdout(df: pd.DataFrame) -> tuple:
    """
    Aplica a técnica de divisão Hold-out estratificado
    70%-30% sobre a base de dados.

    Parametros:
    - df: DataFrame da base de dados

    Retorna:
    - x_train: features da base de treino
    - x_test: features da base de teste
    - y_train: target da base de treino
    - y_test: target da base do teste
    """
    features = ['Acidez fixa',
                'Acidez volátil',
                'Ácido cítrico',
                'Açúcar residual',
                'Cloretos',
                'Dióxido de enxofre livre',
                'Dióxido de enxofre total',
                'Densidade',
                'pH',
                'Sulfatos',
                'Álcool',
                'Qualidade']

    target = 'Tipo'
    
    X = df.drop(target, axis=1)
    Y = df[target]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, 
                                                        test_size=0.3,
                                                        stratify=Y)
                                                        
    
    train_test_set = x_train, x_test, y_train, y_test
        
    return train_test_set


def scaling(train: pd.DataFrame, test: pd.DataFrame):
    """
    Realiza a normalização das features
    da base de dados.

    Parâmetros:
    - train: features da base de treino
    - test: features da base de teste

    Retorna:
    - x_train_scaled: features da base de treino normalizadas
    - x_test_scaled: features da base de teste normalizadas
    """
    scaler = StandardScaler().fit(train)
    x_train_scaled = scaler.transform(train)
    x_test_scaled = scaler.transform(test)
    
    return x_train_scaled, x_test_scaled
    

def model_training(arch: dict, x: pd.DataFrame, y: pd.DataFrame):
    """
    Treina o modelo de rede neural MLPClassifier.

    Parâmetros:
    - arch: Dicionário com parâmetros para a rede neural
    - x: Features da base (teste ou treino)
    - y: Target da base (teste ou treino)

    Retorna:
    - model: Modelo treinado da rede neural
    """
    mlp_clf = MLPClassifier(max_iter = arch["max_iter"], 
                            hidden_layer_sizes = arch["hidden_layer_sizes"],
                            solver = arch["solver"],
                            activation = arch["activation"])
    
    model = mlp_clf.fit(x, y)
    
    return model


###Resultados

In [68]:
def get_results() -> pd.DataFrame:
    """
    Realiza testes de execução com as arquiteturas
    e registra métricas relativas ao 
    desempenho da rede neural na classificação
    dos dados.

    Retorna:
    - df_measures: DataFrame com métricas dos testes
    """

    num_executions = 10
    bases = ["Base de treino", "Base de teste"]
    iterations = list(range(num_executions))

    architectures = ["Tipo 1", "Tipo 2"]
    measures_labels = ["Matriz de confusão",
                        "Sensibilidade",
                        "Especificidade",
                        "Confiabilidade positiva",
                        "Confiabilidade negativa",
                        "Acurácia total"
        ]

    rows_multi_index = pd.MultiIndex.from_product((bases, iterations), names=["Base", "Execução"])
    columns_multi_index = pd.MultiIndex.from_product((architectures, measures_labels), names=["Arquitetura", "Medida"])

    df_measures = pd.DataFrame(index=rows_multi_index, columns=columns_multi_index)
    
    for i in range(num_executions):
        for arch in nn_architectures:

            x_train, x_test, y_train, y_test = apply_stratified_holdout(df)
            x_train, x_test = scaling(x_train, x_test) 

            trained_model = model_training(arch, x_train, y_train)

            features_dict = {"Base de treino": x_train, 
                             "Base de teste": x_test}
            
            target_dict = {"Base de treino": y_train,
                           "Base de teste": y_test}

            for base in bases:
                y_pred = trained_model.predict(features_dict[base])
                
                confusion_matrix = metrics.confusion_matrix(target_dict[base], y_pred)
                true_negatives, false_positives, false_negatives, true_positives = confusion_matrix.ravel()
                
                sensitivity = true_positives / (true_positives + false_negatives)
                specificity = true_negatives / (true_negatives + false_positives)
                positive_predictive_value = true_positives / (true_positives + false_positives)
                negative_predictive_value = true_negatives / (true_negatives + false_negatives)
                accuracy = metrics.accuracy_score(target_dict[base], y_pred)
                
                measures = np.array([sensitivity, 
                                     specificity, 
                                     positive_predictive_value, 
                                     negative_predictive_value, 
                                     accuracy])
                
                df_measures.loc[(base, i), 
                               (f"Tipo {nn_architectures.index(arch) + 1}", 
                                "Matriz de confusão")] = str(np.matrix(confusion_matrix)).replace("\n", ",").replace("[ ", "[")
                
                df_measures.loc[(base, i), (f"Tipo {nn_architectures.index(arch) + 1}", measures_labels[1:])] = measures
                
            for label in measures_labels[1:]:
                df_measures.loc[:, (f"Tipo {nn_architectures.index(arch) + 1}", label)] = pd.to_numeric(
                    df_measures.loc[:, (f"Tipo {nn_architectures.index(arch) + 1}", label)])
        
    return df_measures


if __name__ == '__main__':
    
    df = get_data()

    nn_architectures = [
                    
        {"max_iter" : 100,
         "hidden_layer_sizes" : (10),
         "solver" : "lbfgs",
         "activation" : "relu"},

        {"max_iter" : 300,
         "hidden_layer_sizes" : (10,10,10),
         "solver" : "adam",
         "activation" : "logistic"}

    ]  

    results = get_results()      
    display(results)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

Arquitetura                                 Tipo 1                \
Medida                          Matriz de confusão Sensibilidade   
Base           Execução                                            
Base de treino 0         [[3427    1], [  1 1118]]      0.999106   
               1         [[3428    0], [  1 1118]]      0.999106   
               2         [[3428    0], [  2 1117]]      0.998213   
               3         [[3428    0], [  0 1119]]      1.000000   
               4         [[3428    0], [  2 1117]]      0.998213   
               5         [[3428    0], [  0 1119]]      1.000000   
               6         [[3426    2], [  0 1119]]      1.000000   
               7         [[3428    0], [  0 1119]]      1.000000   
               8         [[3427    1], [  1 1118]]      0.999106   
               9         [[3428    0], [  1 1118]]      0.999106   
Base de teste  0         [[1465    5], [  4  476]]      0.991667   
               1         [[1454   16], [  6  474]]      0.987500   
               2         [[1456   14], [  4  476]]      0.991667   
               3         [[1464    6], [  2  478]]      0.995833   
               4         [[1468    2], [  4  476]]      0.991667   
               5         [[1465    5], [  3  477]]      0.993750   
               6         [[1466    4], [  5  475]]      0.989583   
               7         [[1463    7], [  5  475]]      0.989583   
               8         [[1465    5], [  5  475]]      0.989583   
               9         [[1461    9], [  8  472]]      0.983333   

Arquitetura                                                     \
Medida                  Especificidade Confiabilidade positiva   
Base           Execução                                          
Base de treino 0              0.999708                0.999106   
               1              1.000000                1.000000   
               2              1.000000                1.000000   
               3              1.000000                1.000000   
               4              1.000000                1.000000   
               5              1.000000                1.000000   
               6              0.999417                0.998216   
               7              1.000000                1.000000   
               8              0.999708                0.999106   
               9              1.000000                1.000000   
Base de teste  0              0.996599                0.989605   
               1              0.989116                0.967347   
               2              0.990476                0.971429   
               3              0.995918                0.987603   
               4              0.998639                0.995816   
               5              0.996599                0.989627   
               6              0.997279                0.991649   
               7              0.995238                0.985477   
               8              0.996599                0.989583   
               9              0.993878                0.981289   

Arquitetura                                                     \
Medida                  Confiabilidade negativa Acurácia total   
Base           Execução                                          
Base de treino 0                       0.999708       0.999560   
               1                       0.999708       0.999780   
               2                       0.999417       0.999560   
               3                       1.000000       1.000000   
               4                       0.999417       0.999560   
               5                       1.000000       1.000000   
               6                       1.000000       0.999560   
               7                       1.000000       1.000000   
               8                       0.999708       0.999560   
               9                       0.999708       0.999780   
Base de teste  0                       0.997277       0.99

In [70]:
def get_average_std():
    """
    Calcula média e desvio-padrão das acurácias
    totais das duas arquiteturas tanto para a 
    base de treino quanto para a de teste.

    Retorna:
    - df_average_std: DataFrame com as médias e desvios-padrão 
                      das acurácias totais
    """
    rows = ["Base de treino", "Base de teste"]
    columns = ["Tipo 1", "Tipo 2"]

    accuracy_mean_1_train = results.loc[(rows[0]), ("Tipo 1", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_1_train = results.loc[(rows[0]), ("Tipo 1", ["Acurácia total"])].std().to_numpy()

    accuracy_mean_1_test = results.loc[(rows[1]), ("Tipo 1", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_1_test = results.loc[(rows[1]), ("Tipo 1", ["Acurácia total"])].std().to_numpy()

    accuracy_mean_2_train = results.loc[(rows[0]), ("Tipo 2", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_2_train = results.loc[(rows[0]), ("Tipo 2", ["Acurácia total"])].std().to_numpy()

    accuracy_mean_2_test = results.loc[(rows[1]), ("Tipo 2", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_2_test = results.loc[(rows[1]), ("Tipo 2", ["Acurácia total"])].std().to_numpy()

    cell_11 = f"{accuracy_mean_1_train * 100}% ± {accuracy_std_1_train * 100}%"
    cell_12 = f"{accuracy_mean_1_test * 100}% ± {accuracy_std_1_test * 100}%"
    cell_21 = f"{accuracy_mean_2_train* 100}% ± {accuracy_std_2_train * 100}%"
    cell_22 = f"{accuracy_mean_2_test* 100}% ± {accuracy_std_2_test * 100}%"

    acc_data = [[cell_11, cell_12],
               [cell_21, cell_22]]

    df_average_std = pd.DataFrame(acc_data,index=rows, columns=columns, dtype=np.object_)

    df_average_std.index.name = "Base"
    df_average_std.columns.name = "Arquitetura"

               
    return df_average_std

get_average_std()

Arquitetura,Tipo 1,Tipo 2
Base,,
Base de treino,[99.97360897]% ± [0.02020973]%,[99.38974359]% ± [0.27076621]%
Base de teste,[99.68770618]% ± [0.04374]%,[99.44615385]% ± [0.1426096]%
